# RAG Basics

In [4]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import * as pdfParser from "https://deno.land/x/pdf_parser@v1.1.2/main.js";


const keys = await mod.load({export: true})

In [5]:
import {
    Document,
    VectorStoreIndex,
    SimpleDirectoryReader
} from "npm:llamaindex@0.1.8"

In [7]:
const documents = await new SimpleDirectoryReader()
    .loadData({directoryPath: "../data"})

In [8]:
const index   = await VectorStoreIndex.fromDocuments(documents)

### Create a Query Engine
Conveneience Function
- Retriever
- Postprocessing
- Synthesizer

In [9]:
const queryEngine = index.asQueryEngine()

In [13]:
const response = await queryEngine.query({
    query: "What is love?"
})

In [14]:
console.log(response.toString())

Love, as described in the context information, is a powerful force that dissolves tangles, risks everything, and can effortlessly reduce itself to rubble. It is different from intellect, which is cautious and advises against too much excitement. Love is about embracing imperfections and defects in others, understanding and appreciating them, and ultimately leading to wisdom and a deeper connection with God. It is a force that transcends language and can only be truly understood through silence. Love is about letting life live through you, embracing change, and being open to the unknown. It is about seeing God in everything and everyone, and ultimately, it is about discovering the treasures hidden among ruins, as a broken heart hides treasures.


In [15]:
const response = await queryEngine.query({
    query: "What is 10th rule of love?"
})

In [16]:
console.log(response.toString())

Rule 13: Try not to resist the changes, which come your way. Instead let life live through you. And do not worry that your life is turning upside down. How do you know that the side you are used to is better than the one to come? – Shams, to a novice


What happened?
- LLM to answer questions
- Embedding models to encode them

In [18]:
import * as llamaIndex from "npm:llamaindex@0.1.8"

In [19]:
let customLLM = new llamaIndex.OpenAI()
let customEmbedding = new llamaIndex.OpenAIEmbedding()

In [20]:
let customServiceContext = new llamaIndex.serviceContextFromDefaults({
    llm: customLLM,
    embedModel: customEmbedding
})

### Our own prompt

In [23]:
let customQaPrompt = function({ context= "", query = ""}) {
    return `Context information is below.
        ---------------------
        ${context}
        ---------------------
        Given the context information, answer the query.
        Include a random fact about whales in your answer.\
        The whale fact can come from your training data.
        Query: ${query}
        Answer:`
}

In [24]:
let customResponseBuilder = new llamaIndex.SimpleResponseBuilder(
    customServiceContext,
    customQaPrompt
)

In [25]:
let customSynthesizer = new llamaIndex.ResponseSynthesizer({
    responseBuilder: customResponseBuilder,
    serviceContext: customServiceContext
})

In [26]:
let customRetriever = new llamaIndex.VectorIndexRetriever({
    index
})

In [27]:
let customQueryEngine = new llamaIndex.RetrieverQueryEngine(
    customRetriever,
    customSynthesizer
)

In [28]:
let response2 = await customQueryEngine.query({
    query: "What is love?"
})

In [29]:
console.log(response2.toString())

Love, according to the teachings of Shams, is the ultimate key to understanding and knowing God. It is not just about loving a perfect deity, but also about embracing the imperfections and quirks of fellow human beings. True wisdom, as Shams suggests, cannot exist without love for God's creation. In essence, love is the bridge that connects us to both humanity and divinity, allowing us to truly know and appreciate the beauty of the world around us.

Random whale fact: Whales are known for their complex communication skills, using a variety of sounds and vocalizations to communicate with each other over long distances in the ocean. This ability to communicate effectively is crucial for their social interactions and survival in their marine environment.
